In [1]:
from easyocr import Reader
import torch
from dotenv import load_dotenv
print(torch.xpu.is_available())

True


In [2]:
if torch.xpu.is_available():
    print("Intel XPU is available!")
    device_str = 'xpu'
    # Optional: print XPU device name
    print(f"Using device: {torch.xpu.get_device_name(0)}")
else:
    print("Intel XPU not available, falling back to CPU.")
    device_str = 'cpu'

reader = Reader(['en'], gpu=True)

reader.device = torch.device(device_str)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Intel XPU is available!
Using device: Intel(R) Arc(TM) Graphics


In [3]:
# 3. If XPU is available, manually move models and set the device for EasyOCR
if device_str == 'xpu':
    print(f"Attempting to move EasyOCR models to {device_str}...")
    try:
        # Set the device attribute that EasyOCR uses internally
        reader.device = torch.device(device_str) # Use torch.device object

        # Move the detector model to XPU
        if hasattr(reader, 'detector') and reader.detector is not None:
            print("Moving detector model to XPU...")
            reader.detector = reader.detector.to(reader.device)
            print("Detector model moved.")
        else:
            print("Reader does not have a 'detector' attribute or it's None.")


        # Move the recognizer model to XPU
        # The recognizer model itself is reader.recognizer
        # Its sub-modules might also need explicit moving if .to(device) doesn't recurse fully
        # for all custom structures, but usually top-level .to() is enough for nn.Module.
        if hasattr(reader, 'recognizer') and reader.recognizer is not None:
            print("Moving recognizer model to XPU...")
            reader.recognizer = reader.recognizer.to(reader.device)

            # EasyOCR's 'standard' recognizer has sub-modules like 'character_extractor',
            # 'feature_extraction', 'SequenceModeling', 'Prediction'.
            # A single .to(device) on the parent nn.Module (reader.recognizer)
            # should move all its parameters and buffers.
            # Let's confirm if this is sufficient or if we need to move submodules explicitly.
            # For most PyTorch nn.Modules, parent.to(device) is enough.

            print("Recognizer model moved.")
        else:
            print("Reader does not have a 'recognizer' attribute or it's None.")

        print(f"EasyOCR models and internal device set to: {reader.device}")
        print("Successfully configured EasyOCR for XPU.")

    except Exception as e:
        print(f"Error moving models to XPU: {e}")
        print("Falling back to CPU for EasyOCR.")
        # Re-initialize on CPU or ensure it's on CPU
        reader = Reader(['en'], gpu=False, verbose=True)
        device_str = 'cpu'
else:
    print("EasyOCR will run on CPU.")

Attempting to move EasyOCR models to xpu...
Moving detector model to XPU...
Detector model moved.
Moving recognizer model to XPU...
Recognizer model moved.
EasyOCR models and internal device set to: xpu
Successfully configured EasyOCR for XPU.


In [4]:
while True:
    try:
        # Example usage: read text from an image file
        result = reader.readtext('test_image.jpg')
        print("OCR Result:", result)
    except Exception as e:
        print(f"Error during OCR processing: {e}")
    
    # Break the loop after one iteration for demonstration purposes
    break

OCR Result: [([[np.int32(46), np.int32(98)], [np.int32(584), np.int32(98)], [np.int32(584), np.int32(218)], [np.int32(46), np.int32(218)]], 'GRAUSCALE', np.float64(0.7877132613204044)), ([[np.int32(52), np.int32(214)], [np.int32(244), np.int32(214)], [np.int32(244), np.int32(246)], [np.int32(52), np.int32(246)]], 'Fully Editable PSD', np.float64(0.9652309234689471)), ([[np.int32(308), np.int32(214)], [np.int32(574), np.int32(214)], [np.int32(574), np.int32(246)], [np.int32(308), np.int32(246)]], 'Made with Smart Objects', np.float64(0.9476364351749266))]
